<a href="https://colab.research.google.com/github/YoonseoOh/Introductory-into-Machine-Learning/blob/main/midproject_20221061_Yoonseo_Oh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml("mnist_784")

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [32]:
X, y = mnist.data.values, mnist.target.values
# X = X.astype('float32')
# y = y.astype('int')

In [33]:
#Checking pixel configuration for Bernoulli operations
import numpy as np

X_check = mnist.data.to_numpy()

# Calculate unique pixel values
unique_values = np.unique(X_check)

# Check the range of pixel values
min_pixel_value = np.min(unique_values)
max_pixel_value = np.max(unique_values)

print("Minimum Unique Pixel Value:", min_pixel_value)
print("Maximum Unique Pixel Value:", max_pixel_value)

Minimum Unique Pixel Value: 0.0
Maximum Unique Pixel Value: 255.0


In [34]:
# Convert to binary data
X_binary = (X > 128).astype(int)

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Data segmentation
X_train, X_test, y_train, y_test = train_test_split(X_binary, y, test_size=0.2, random_state=13)

In [36]:
# Bernoulli Naive Bayes Model Class
class BernoulliNBWithLog:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.class_priors = np.array([np.log(np.mean(y == c)) for c in self.classes])

        # Calculate the probability of each characteristic by class
        self.feature_probs = []
        for c in self.classes:
            feature_prob = (X[y == c].sum(axis=0)) / (np.sum(y == c))
            self.feature_probs.append(np.log(np.clip(feature_prob, 1e-10, 1.0 - 1e-10)))
        self.feature_probs = np.array(self.feature_probs)

    def predict(self, X):
        # Use log-transformed characteristic probabilities and log prior probabilities to calculate log-post probabilities
        log_likelihoods = np.dot(X, self.feature_probs.T)
        log_posteriors = log_likelihoods + self.class_priors

        # Select the class with the highest log-post probability
        predicted_class = self.classes[np.argmax(log_posteriors, axis=1)]
        return predicted_class

In [37]:
# Model initialization and training
model = BernoulliNBWithLog()

In [38]:
model.fit(X_train, y_train)


In [39]:
# Evaluate the model with test data
y_pred = model.predict(X_test)

In [40]:
# Accuracy calculation
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 69.51%
